## RAG pipeline using  Components *(LlamaIndex)*

#### Loading and Embedding Documents

In [46]:
!pip install -Uqq pip

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
##! Installing dependencies 

!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api  llama-index-embeddings-huggingface -U -q

Use Cases of these dependencies *(explaining just unusual ones)*:

- `llama-index-callbacks-arize-phoenix`:  Integrates LlamaIndex with arise phoenix; a tool desined for oberservability in LLM applications
- `llama-index-vector-stores-chroma`: Integrates LlamaIndex with ChromaDB; a vector database that stores embeddings for effecient similarity based querying

In [11]:
#@ Extra step required in kaggle (haiyaa)
!git config --global credential.helper store

In [13]:
from huggingface_hub import login

login()

#### Creating a `QueryEngine` for RAG:

Dataset being used: [dvilasuero/finepersonas-v0.1-tiny](https://huggingface.co/datasets/dvilasuero/finepersonas-v0.1-tiny)

In [15]:
from datasets import load_dataset
from pathlib import Path

ds = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(ds):
    with open(Path("data")/f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

In [40]:
##@ Inspecting a persona
!cat persona_100.txt

An environmental conservationist focused on wetland ecosystems and their role in mitigating climate change.

In [38]:
##@ Loading and embedding 
from llama_index.core import SimpleDirectoryReader #@ Used for local files


reader = SimpleDirectoryReader(input_dir = "/kaggle/working/data")
documents = reader.load_data()
len(documents)

5000

In [64]:
##@ Creating the ingestionpipeline to create nodes from the documents and prepare them for the `QueryEngine`

from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# Create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
    ]
)

# Pipeline Run 
nodes = await pipeline.arun(documents = documents[:10])
nodes

/tmp/ipykernel_31/3376189987.py:11: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")


[TextNode(id_='c4b1d2a0-b77d-4ae7-91bc-d27af9688ac9', embedding=[0.0005078223184682429, 0.008853902108967304, 0.04555211961269379, 0.021935340017080307, -0.01887446828186512, -0.016758069396018982, 0.025069579482078552, 0.030929474160075188, -0.07692992687225342, -0.04644887149333954, 0.0008292985148727894, -0.03204047679901123, -0.059489816427230835, 0.027530472725629807, 0.00045563685125671327, -0.0076825376600027084, -0.009479020722210407, 0.09515306353569031, 0.018469104543328285, 0.008889887481927872, 0.022798199206590652, -0.05350906774401665, 0.026824891567230225, -0.012373792938888073, -0.032322611659765244, 0.019309693947434425, 0.02455553226172924, -0.0053581190295517445, -0.009278228506445885, -0.1091209203004837, -0.08289098739624023, 0.005203999113291502, -0.025418639183044434, 0.002742330776527524, 0.05239914357662201, 0.01319335401058197, -0.0006513189873658121, 0.07388011366128922, 0.0014130141353234649, 0.010451330803334713, 0.045067042112350464, 0.020180189982056618, 

#### Storing and indexing documents

In [65]:
##@ Since we are using the ingestion pipeline, we can directly attach a vector store to the pipeline to populate it. 

import chromadb  
from llama_index.vector_stores.chroma import ChromaVectorStore

db= chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection= chroma_collection)

pipeline = IngestionPipeline(
    transformations= [
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
    ],
    vector_store = vector_store,
)

nodes = await pipeline.arun(documents= documents[:10])

len(nodes)

/tmp/ipykernel_31/4019586733.py:13: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")


10

In [68]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbedding


embed_model = HuggingFaceInferenceAPIEmbedding(model_name = "BAAI/bge-small-en-v1.5")
index= VectorStoreIndex.from_vector_store(
    vector_store = vector_store, embed_model=embed_model
)

/tmp/ipykernel_31/1671654127.py:5: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  embed_model = HuggingFaceInferenceAPIEmbedding(model_name = "BAAI/bge-small-en-v1.5")


> **ⓘ Note:** \
> `from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding` is depreciated use `from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbedding` instead

In [75]:
#@ Querying the index

from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import nest_asyncio           ## To run the query engine

nest_asyncio.apply()

llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine = index.as_query_engine(
    llm = llm,
    response_mode = "tree_summarize"
)
response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)

response

Response(response="The author is an anthropologist or cultural expert deeply immersed in the study of Cypriot culture, history, and society. With extensive research and firsthand experience living in Cyprus, this individual has cultivated a profound understanding of the local people, customs, and way of life. Their expertise is rooted in the nuances of Cypriot traditions, making them a valuable resource for those seeking insight into the island's rich cultural heritage.", source_nodes=[NodeWithScore(node=TextNode(id_='e5c44671-c67a-4f0d-b8f2-e1bee9ae4234', embedding=None, metadata={'file_path': '/kaggle/working/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-04-13', 'last_modified_date': '2025-04-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'l

#### Evaluation and Observability

In [79]:
from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response= response)

eval_result.passing

True

In [80]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
llama_trace_token = user_secrets.get_secret("Llama_trace_phoenix")

In [83]:
!pip install -q arize-phoenix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [105]:
import llama_index
import os

PHOENIX_API_KEY = "<PHOENIX_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://app.phoenix.arize.com"
)

In [106]:
response = query_engine.query(
    "What is the name of the someone that is interested in AI and techhnology?"
)
response

Response(response='The provided information does not mention anyone interested in AI and technology. The description focuses on an anthropologist or cultural expert with a deep understanding of Cypriot culture, history, and society. There is no reference to AI or technology in the given context.', source_nodes=[NodeWithScore(node=TextNode(id_='e5c44671-c67a-4f0d-b8f2-e1bee9ae4234', embedding=None, metadata={'file_path': '/kaggle/working/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-04-13', 'last_modified_date': '2025-04-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='624e3f01-99be-47d8-9550-9951519498b7', node_type='4', metadata=